[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/williamyang1991/VToonify/blob/master/notebooks/inference_playground.ipynb)

This colab contains three parts


- **PART1**: Build a web demo with Gradio UI for easy use
  - Easy Uploading of your own photo or video
  - Easy loading of pre-trained model
  - Just click the button to toonify
  - The pretrained model will be downloaded from HuggingFace model
  - This part is independent of the following two parts.
- **PART2**: Style tranfer with Colab UI where you can look into the code details and easily modify the code
  - The pretrained model will be downloaded from Google Drive
- **PART3**: Style control with Colab UI where you can look into the code details and easily modify the code
  - The pretrained model will be downloaded from Google Drive

# PART I - Style Transfer with Gradio UI


In [ ]:
!pip install gradio huggingface_hub

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.chdir('../')
DEMO_DIR = 'VToonify_Gradio'
device = 'cuda'

In [ ]:
!git clone https://huggingface.co/spaces/PKUWilliamYang/VToonify $DEMO_DIR

In [ ]:
os.chdir(f'./{DEMO_DIR}')

In [ ]:
from __future__ import annotations
import pathlib
import gradio as gr
from vtoonify_model import Model

def update_slider(choice: str) -> dict:
    if type(choice) == str and choice.endswith('-d'):
        return gr.Slider.update(maximum=1, minimum=0, value=0.5)
    else:
        return gr.Slider.update(maximum=0.5, minimum=0.5, value=0.5)

def set_example_image(example: list) -> dict:
    return gr.Image.update(value=example[0])

def set_example_video(example: list) -> dict:
    return gr.Video.update(value=example[0]), 
   
sample_video = ['./vtoonify/data/529_2.mp4','./vtoonify/data/7154235.mp4','./vtoonify/data/651.mp4','./vtoonify/data/908.mp4']
sample_vid = gr.Video(label='Video file')  #for displaying the example
example_videos = gr.components.Dataset(components=[sample_vid], samples=[[path] for path in sample_video], type='values', label='Video Examples')     

model = Model('cuda')

# if you would like to run long videos, set model.video_limit_gpu to a large value
model.video_limit_gpu = 1000000   # currently we don't have limitation on the frames with GPU
model.video_limit_cpu = 100   # currently we only toonify 100 frames per video with CPU

In [ ]:
DESCRIPTION = '''
<div align=center>
<h1 style="font-weight: 900; margin-bottom: 7px;">
   Portrait Style Transfer with <a href="https://github.com/williamyang1991/VToonify">VToonify</a>
</h1>
</div>
'''
FOOTER = '<div align=center><img id="visitor-badge" alt="visitor badge" src="https://visitor-badge.laobi.icu/badge?page_id=williamyang1991/VToonify" /></div>'

ARTICLE = r"""
If VToonify is helpful, please help to ⭐ the <a href='https://github.com/williamyang1991/VToonify' target='_blank'>Github Repo</a>. Thanks! 
[![GitHub Stars](https://img.shields.io/github/stars/williamyang1991/VToonify?style=social)](https://github.com/williamyang1991/VToonify)
---
📝 **Citation**
If our work is useful for your research, please consider citing:
```bibtex
@article{yang2022Vtoonify,
  title={VToonify: Controllable High-Resolution Portrait Video Style Transfer},
  author={Yang, Shuai and Jiang, Liming and Liu, Ziwei and Loy, Chen Change},
  journal={ACM Transactions on Graphics (TOG)},
  volume={41},
  number={6},
  articleno={203},
  pages={1--15},
  year={2022},
  publisher={ACM New York, NY, USA},
  doi={10.1145/3550454.3555437},
}
```
📋 **License**
This project is licensed under <a rel="license" href="https://github.com/williamyang1991/VToonify/blob/main/LICENSE.md">S-Lab License 1.0</a>. 
Redistribution and use for non-commercial purposes should follow this license.
📧 **Contact**
If you have any questions, please feel free to reach me out at <b>williamyang@pku.edu.cn</b>.
"""


with gr.Blocks() as demo:

    gr.Markdown(DESCRIPTION)

    with gr.Box():
        gr.Markdown('''## Step 1(Select Style)
- Select **Style Type**.
    - Type with `-d` means it supports style degree adjustment.
    - Type without `-d` usually has better toonification quality.
''')
        with gr.Row():
            with gr.Column():
                gr.Markdown('''Select Style Type''')  
                with gr.Row():
                    style_type = gr.Radio(label='Style Type',
                                          choices=['cartoon1','cartoon1-d','cartoon2-d','cartoon3-d',
                                                   'cartoon4','cartoon4-d','cartoon5-d','comic1-d',
                                                   'comic2-d', 'arcane1','arcane1-d','arcane2', 'arcane2-d',
                                                   'caricature1','caricature2','pixar','pixar-d',
                                                   'illustration1-d', 'illustration2-d', 'illustration3-d', 'illustration4-d', 'illustration5-d', 
                                                  ]
                                         )   
                    exstyle = gr.Variable()
                with gr.Row():
                    loadmodel_button = gr.Button('Load Model')
                with gr.Row():
                    load_info = gr.Textbox(label='Process Information', interactive=False, value='No model loaded.')
            with gr.Column():
                gr.Markdown('''Reference Styles
![example](https://raw.githubusercontent.com/williamyang1991/tmpfile/master/vtoonify/style.jpg)''')   


    with gr.Box():
        gr.Markdown('''## Step 2 (Preprocess Input Image / Video)
- Drop an image/video containing a near-frontal face to the **Input Image**/**Input Video**.
- Hit the **Rescale Image**/**Rescale First Frame** button.
    - Rescale the input to make it best fit the model.
    - The final image result will be based on this **Rescaled Face**. Use padding parameters to adjust the background space.
    - **<font color=red>Solution to [Error: no face detected!]</font>**: VToonify uses dlib.get_frontal_face_detector but sometimes it fails to detect a face. You can try several times or use other images until a face is detected, then switch back to the original image.
- For video input, further hit the **Rescale Video** button.
    - The final video result will be based on this **Rescaled Video**. To avoid overload, video is cut to at most **100** frames for CPU, respectively.
''')
        with gr.Row():
            with gr.Box():
                with gr.Column():
                    gr.Markdown('''Choose the padding parameters.
    ![example](https://raw.githubusercontent.com/williamyang1991/tmpfile/master/vtoonify/rescale.jpg)''')
                    with gr.Row():
                        top = gr.Slider(128,
                                        256,
                                        value=160,
                                        step=8,
                                        label='top')
                    with gr.Row():
                        bottom = gr.Slider(128,
                                        256,
                                        value=160,
                                        step=8,
                                        label='bottom')
                    with gr.Row():
                        left = gr.Slider(128,
                                        256,
                                        value=160,
                                        step=8,
                                        label='left')
                    with gr.Row():
                        right = gr.Slider(128,
                                        256,
                                        value=160,
                                        step=8,
                                        label='right')     
            with gr.Box():
                with gr.Column():
                    gr.Markdown('''Input''')                
                    with gr.Row():
                        input_image = gr.Image(label='Input Image',
                                                       type='filepath')
                    with gr.Row():
                        preprocess_image_button = gr.Button('Rescale Image') 
                    with gr.Row():
                        input_video = gr.Video(label='Input Video',
                                               mirror_webcam=False,
                                                      type='filepath')  
                    with gr.Row():
                        preprocess_video0_button = gr.Button('Rescale First Frame')
                        preprocess_video1_button = gr.Button('Rescale Video')

            with gr.Box():
                with gr.Column():
                    gr.Markdown('''View''')
                    with gr.Row():
                        input_info = gr.Textbox(label='Process Information', interactive=False, value='n.a.')
                    with gr.Row():
                        aligned_face = gr.Image(label='Rescaled Face',
                                        type='numpy',
                                        interactive=False)
                        instyle = gr.Variable()
                    with gr.Row():
                        aligned_video = gr.Video(label='Rescaled Video',
                                        type='mp4',
                                        interactive=False)  
        with gr.Row():
            with gr.Column():
                paths = ['./vtoonify/data/pexels-andrea-piacquadio-733872.jpg','./vtoonify/data/i5R8hbZFDdc.jpg','./vtoonify/data/yRpe13BHdKw.jpg','./vtoonify/data/ILip77SbmOE.jpg','./vtoonify/data/077436.jpg','./vtoonify/data/081680.jpg']
                example_images = gr.Dataset(components=[input_image],
                                        samples=[[path] for path in paths],
                                           label='Image Examples')
            with gr.Column():
                #example_videos = gr.Dataset(components=[input_video], samples=[['./vtoonify/data/529.mp4']], type='values')
                #to render video example on mouse hover/click        
                example_videos.render()
                #to load sample video into input_video upon clicking on it
                def load_examples(video):  
                    #print("****** inside load_example() ******")
                    #print("in_video is : ", video[0])
                    return video[0]

                example_videos.click(load_examples, example_videos, input_video) 

    with gr.Box():
        gr.Markdown('''## Step 3 (Generate Style Transferred Image/Video)''')
        with gr.Row():
            with gr.Column():
                gr.Markdown('''
                    - Adjust **Style Degree**.
                    - Hit **Toonify!** to toonify one frame. Hit **VToonify!** to toonify full video.
                        - Estimated time on 1600x1440 video of 300 frames: 1 hour (CPU); 2 mins (GPU)
                    ''')
                style_degree = gr.Slider(0,
                                         1,
                                         value=0.5,
                                         step=0.05,
                                         label='Style Degree')  
            with gr.Column():
                gr.Markdown('''![example](https://raw.githubusercontent.com/williamyang1991/tmpfile/master/vtoonify/degree.jpg)
                    ''')  
        with gr.Row():
            output_info = gr.Textbox(label='Process Information', interactive=False, value='n.a.')
        with gr.Row():
            with gr.Column():
                with gr.Row():
                    result_face = gr.Image(label='Result Image',
                                        type='numpy',
                                        interactive=False)
                with gr.Row():
                    toonify_button = gr.Button('Toonify!')
            with gr.Column():
                with gr.Row():
                    result_video = gr.Video(label='Result Video',
                                        type='mp4',
                                        interactive=False)    
                with gr.Row():
                    vtoonify_button = gr.Button('VToonify!')

    gr.Markdown(ARTICLE)
    gr.Markdown(FOOTER)

    loadmodel_button.click(fn=model.load_model,
                            inputs=[style_type],
                            outputs=[exstyle, load_info])


    style_type.change(fn=update_slider,
                      inputs=style_type,
                      outputs=style_degree)

    preprocess_image_button.click(fn=model.detect_and_align_image,
                            inputs=[input_image, top, bottom, left, right],
                            outputs=[aligned_face, instyle, input_info])
    preprocess_video0_button.click(fn=model.detect_and_align_video,
                            inputs=[input_video, top, bottom, left, right],
                            outputs=[aligned_face, instyle, input_info])
    preprocess_video1_button.click(fn=model.detect_and_align_full_video,
                            inputs=[input_video, top, bottom, left, right],
                            outputs=[aligned_video, instyle, input_info])

    toonify_button.click(fn=model.image_toonify,
                            inputs=[aligned_face, instyle, exstyle],
                            outputs=[result_face, output_info])
    vtoonify_button.click(fn=model.video_tooniy,
                            inputs=[aligned_video, instyle, exstyle],
                            outputs=[result_video, output_info])
    example_images.click(fn=set_example_image,
                                 inputs=example_images,
                                 outputs=example_images.components)

In [ ]:
# launch a web demo, best open the public URL and view on new web page
demo.launch(
    debug=True,
)

In [ ]:
# you can close the web demo by running
demo.close()

# PART II - Style Transfer with specialized VToonify-D model

code is mainly modified from [pixel2style2pixel](https://github.com/eladrich/pixel2style2pixel/blob/master/notebooks/inference_playground.ipynb)

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.chdir('../')
CODE_DIR = 'VToonify'
device = 'cuda'

In [ ]:
!git clone https://github.com/williamyang1991/VToonify.git $CODE_DIR

In [ ]:
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force 
!pip install wget

In [ ]:
os.chdir(f'./{CODE_DIR}')
MODEL_DIR = os.path.join(os.path.dirname(os.getcwd()), CODE_DIR, 'checkpoint')
DATA_DIR = os.path.join(os.path.dirname(os.getcwd()), CODE_DIR, 'data')
OUT_DIR = os.path.join(os.path.dirname(os.getcwd()), CODE_DIR, 'output')

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(".")
sys.path.append("..")

import argparse
import numpy as np
import cv2
import dlib
import torch
from torchvision import transforms
import torchvision
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt
from model.vtoonify import VToonify
from model.bisenet.model import BiSeNet
from model.encoder.align_all_parallel import align_face
from util import save_image, load_image, visualize, load_psp_standalone, get_video_crop_parameter, tensor2cv2

In [ ]:
def get_download_model_command(file_id, file_name):
    """ Get wget download command for downloading the desired model and save to directory ../checkpoint/. """
    current_directory = os.getcwd()
    save_path = MODEL_DIR
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    url = r"""wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={FILE_ID}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={FILE_ID}" -O {SAVE_PATH}/{FILE_NAME} && rm -rf /tmp/cookies.txt""".format(FILE_ID=file_id, FILE_NAME=file_name, SAVE_PATH=save_path)
    return url

MODEL_PATHS = {
    "encoder": {"id": "1NgI4mPkboYvYw3MWcdUaQhkr0OWgs9ej", "name": "encoder.pt"},
    "faceparsing": {"id": "1jY0mTjVB8njDh6e0LP_2UxuRK3MnjoIR", "name": "faceparsing.pth"},
    "arcane_exstyle": {"id": "1TC67wRJkdmNRZTqYMUEFkrhWRKKZW40c", "name": "exstyle_code.npy"},
    "caricature_exstyle": {"id": "1xr9sx_WmRYJ4qHGTtdVQCSxSo4HP3-ip", "name": "exstyle_code.npy"},
    "cartoon_exstyle": {"id": "1BuCeLk3ASZcoHlbfT28qNru4r5f-hErr", "name": "exstyle_code.npy"},
    "pixar_exstyle": {"id": "1yTaKuSrL7I0i0RYEEK5XD6GI-y5iNUbj", "name": "exstyle_code.npy"},
    "arcane000": {"id": "1pF4fJ8acmawMsjjXo4HXRIOXeZR8jLVh", "name": "generator.pt"},
    "arcane077": {"id": "16rLTF2oC0ZeurnM6hjrfrc8BxtW8P8Qf", "name": "generator.pt"},
    "caricature039": {"id": "1C1E4WEoDWzl0nAxR9okKffFmlMOENbeF", "name": "generator.pt"},
    "caricature068": {"id": "1B1ko1x8fX2aJ4BYCL12AnknVAi3qQc8W", "name": "generator.pt"},
    "cartoon026": {"id": "1YJYODh_vEyUrL0q02okjcicpJhdYY8An", "name": "generator.pt"},
    "cartoon299": {"id": "101qMUMfcI2qDxEbfCBt5mOg2aSqdTaIt", "name": "generator.pt"},
    "pixar052": {"id": "16j_l1x0DD0PjwO8YdplAk69sh3-v95rr", "name": "generator.pt"},
    "cartoon": {"id": "11s0hwhZWTLacMAzZH4OU-o3Qkp54h30J", "name": "generator.pt"},
}

In [ ]:
%matplotlib inline

## Step 1: Select Style Type

In [ ]:
#@title Select a style type { run: "auto", vertical-output: true, display-mode: "both" }
style_type = "cartoon026" #@param ["cartoon026", "cartoon299", "arcane000", "arcane077", "pixar052", "caricature039", "caricature068"]

"""
cartoon026:      balanced 
cartoon299:      big eyes 
arcane000:       for female 
arcane077:       for male 
pixar052:                  
caricature039:   big mouth 
caricature068:   balanced  
"""


## Step 2: Download Pretrained Models 
As part of this repository, we provide pretrained models. We'll download the model and save them to the folder `../checkpoint/`.

In [ ]:
# download pSp encoder and face parsinf network
path = MODEL_PATHS["encoder"]
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"])
!{download_command}
path = MODEL_PATHS["faceparsing"]
download_command = get_download_model_command(file_id=path["id"], file_name=path["name"])
!{download_command}

In [ ]:
# download vtoonify
path = MODEL_PATHS[style_type]
download_command = get_download_model_command(file_id=path["id"], file_name = style_type + '_' + path["name"])
!{download_command}
# download extrinsic style code
path = MODEL_PATHS[style_type[:-3]+'_exstyle']
download_command = get_download_model_command(file_id=path["id"], file_name = style_type[:-3] + '_' + path["name"])
!{download_command}

## Step 3: Load Pretrained Model
We assume that you have downloaded all relevant models and placed them in the directory defined by the above dictionary.

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5,0.5,0.5]),
    ])

vtoonify = VToonify(backbone = 'dualstylegan')
vtoonify.load_state_dict(torch.load(os.path.join(MODEL_DIR, style_type+'_generator.pt'), map_location=lambda storage, loc: storage)['g_ema'])
vtoonify.to(device)

parsingpredictor = BiSeNet(n_classes=19)
parsingpredictor.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'faceparsing.pth'), map_location=lambda storage, loc: storage))
parsingpredictor.to(device).eval()

modelname = './checkpoint/shape_predictor_68_face_landmarks.dat'
if not os.path.exists(modelname):
    import wget, bz2
    wget.download('http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2', modelname+'.bz2')
    zipfile = bz2.BZ2File(modelname+'.bz2')
    data = zipfile.read()
    open(modelname, 'wb').write(data) 
landmarkpredictor = dlib.shape_predictor(modelname)

pspencoder = load_psp_standalone(os.path.join(MODEL_DIR, 'encoder.pt'), device)    

exstyles = np.load(os.path.join(MODEL_DIR, style_type[:-3]+'_exstyle_code.npy'), allow_pickle='TRUE').item()
stylename = list(exstyles.keys())[int(style_type[-3:])]
exstyle = torch.tensor(exstyles[stylename]).to(device)
with torch.no_grad():  
    exstyle = vtoonify.zplus2wplus(exstyle)
    
print('Model successfully loaded!')

## Step 4: Image Toonification

### Visualize and Rescale Input
We rescale the input image to make it fit our pre-trained model

In [ ]:
image_path = './data/077436.jpg'
original_image = load_image(image_path)

In [ ]:
visualize(original_image[0], 30)

In [ ]:
frame = cv2.imread(image_path)
frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

scale = 1
kernel_1d = np.array([[0.125],[0.375],[0.375],[0.125]])
# We detect the face in the image, and resize the image so that the eye distance is 64 pixels.
# Centered on the eyes, we crop the image to almost 400x400 (based on args.padding).
paras = get_video_crop_parameter(frame, landmarkpredictor, padding=[200,200,200,200])
if paras is not None:
    h,w,top,bottom,left,right,scale = paras
    H, W = int(bottom-top), int(right-left)
    # for HR image, we apply gaussian blur to it to avoid over-sharp stylization results
    if scale <= 0.75:
        frame = cv2.sepFilter2D(frame, -1, kernel_1d, kernel_1d)
    if scale <= 0.375:
        frame = cv2.sepFilter2D(frame, -1, kernel_1d, kernel_1d)
    frame = cv2.resize(frame, (w, h))[top:bottom, left:right]
    x = transform(frame).unsqueeze(dim=0).to(device)
else:
    print('no face detected!')

In [ ]:
visualize(x[0].cpu(), 30)

### Perform Inference

In [ ]:
with torch.no_grad():
    I = align_face(frame, landmarkpredictor)
    I = transform(I).unsqueeze(dim=0).to(device)
    s_w = pspencoder(I)
    s_w = vtoonify.zplus2wplus(s_w)
    s_w[:,:7] = exstyle[:,:7]
    # parsing network works best on 512x512 images, so we predict parsing maps on upsmapled frames
    # followed by downsampling the parsing maps
    x_p = F.interpolate(parsingpredictor(2*(F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)))[0], 
                        scale_factor=0.5, recompute_scale_factor=False).detach()
    # we give parsing maps lower weight (1/16)
    inputs = torch.cat((x, x_p/16.), dim=1)
    # d_s has no effect when backbone is toonify
    y_tilde = vtoonify(inputs, s_w.repeat(inputs.size(0), 1, 1), d_s = 0.5)        
    y_tilde = torch.clamp(y_tilde, -1, 1)

In [ ]:
visualize(y_tilde[0].cpu(), 60)

## Step 5: Video Toonification

### Visualize and Rescale Input
We rescale the input video to make it fit our pre-trained model

In [ ]:
video_path = './data/529.mp4'
video_cap = cv2.VideoCapture(video_path)
num = int(video_cap.get(7))

In [ ]:
success, frame = video_cap.read()
if success == False:
    assert('load video frames error')
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

In [ ]:
visualize(transform(frame), 30)

In [ ]:
scale = 1
kernel_1d = np.array([[0.125],[0.375],[0.375],[0.125]])
# We proprocess the video by detecting the face in the first frame, 
# and resizing the frame so that the eye distance is 64 pixels.
# Centered on the eyes, we crop the first frame to almost 400x400 (based on args.padding).
# All other frames use the same resizing and cropping parameters as the first frame.
paras = get_video_crop_parameter(frame, landmarkpredictor, padding=[200,200,200,200])
if paras is None:
    print('no face detected!')
else:
    h,w,top,bottom,left,right,scale = paras
    H, W = int(bottom-top), int(right-left)
# for HR video, we apply gaussian blur to the frames to avoid flickers caused by bilinear downsampling
# this can also prevent over-sharp stylization results. 
if scale <= 0.75:
    frame = cv2.sepFilter2D(frame, -1, kernel_1d, kernel_1d)
if scale <= 0.375:
    frame = cv2.sepFilter2D(frame, -1, kernel_1d, kernel_1d)
frame = cv2.resize(frame, (w, h))[top:bottom, left:right]

In [ ]:
visualize(transform(frame), 30)

### Perform Inference

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
videoWriter = cv2.VideoWriter(os.path.join(OUT_DIR, 'result.mp4'), fourcc, video_cap.get(5), (4*W, 4*H))
batch_size = 4

In [ ]:
with torch.no_grad():
    batch_frames = []
    for i in tqdm(range(num)):
        if i == 0:        
            I = align_face(frame, landmarkpredictor)
            I = transform(I).unsqueeze(dim=0).to(device)
            s_w = pspencoder(I)
            s_w = vtoonify.zplus2wplus(s_w)
            s_w[:,:7] = exstyle[:,:7]
        else:
            success, frame = video_cap.read()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if scale <= 0.75:
                frame = cv2.sepFilter2D(frame, -1, kernel_1d, kernel_1d)
            if scale <= 0.375:
                frame = cv2.sepFilter2D(frame, -1, kernel_1d, kernel_1d)
            frame = cv2.resize(frame, (w, h))[top:bottom, left:right]

        batch_frames += [transform(frame).unsqueeze(dim=0).to(device)]

        if len(batch_frames) == batch_size or (i+1) == num:
            x = torch.cat(batch_frames, dim=0)
            batch_frames = []
            # parsing network works best on 512x512 images, so we predict parsing maps on upsmapled frames
            # followed by downsampling the parsing maps
            x_p = F.interpolate(parsingpredictor(2*(F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)))[0], 
                            scale_factor=0.5, recompute_scale_factor=False).detach()
            # we give parsing maps lower weight (1/16)
            inputs = torch.cat((x, x_p/16.), dim=1)
            # d_s has no effect when backbone is toonify
            y_tilde = vtoonify(inputs, s_w.repeat(inputs.size(0), 1, 1), d_s = 0.5)       
            y_tilde = torch.clamp(y_tilde, -1, 1)
            for k in range(y_tilde.size(0)):
                videoWriter.write(tensor2cv2(y_tilde[k].cpu()))
videoWriter.release()
video_cap.release()

In [ ]:
viz = torchvision.utils.make_grid(y_tilde, 2, 2)
visualize(viz.cpu(), 120)

### Find the stylized video in `./output/result.mp4`

# PART III - Style control with VToonify-Dsd model

## Step 1: Download Pretrained Models 
As part of this repository, we provide pretrained models. We'll download the model and save them to the folder `../checkpoint/`.

In [ ]:
# if you haved downloaded the encoder and faceparsing model in PART I, skip this step
if False:
    path = MODEL_PATHS["encoder"]
    download_command = get_download_model_command(file_id=path["id"], file_name=path["name"])
    !{download_command}
    path = MODEL_PATHS["faceparsing"]
    download_command = get_download_model_command(file_id=path["id"], file_name=path["name"])
    !{download_command}

In [ ]:
# download the style code and the vtoonify-Dsd
path = MODEL_PATHS['cartoon_exstyle']
download_command = get_download_model_command(file_id=path["id"], file_name = 'cartoon_exstyle_code.npy')
!{download_command}
path = MODEL_PATHS['cartoon']
download_command = get_download_model_command(file_id=path["id"], file_name = 'cartoon_generator.pt')
!{download_command}

## Step 2: Load Pretrained Model
We assume that you have downloaded all relevant models and placed them in the directory defined by the above dictionary.

In [ ]:
# if you haved load the models in PART I, skip this step, or set False to True
if True:
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5,0.5,0.5]),
        ])

    parsingpredictor = BiSeNet(n_classes=19)
    parsingpredictor.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'faceparsing.pth'), map_location=lambda storage, loc: storage))
    parsingpredictor.to(device).eval()

    modelname = './checkpoint/shape_predictor_68_face_landmarks.dat'
    if not os.path.exists(modelname):
        wget.download('http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2', modelname+'.bz2')
        zipfile = bz2.BZ2File(modelname+'.bz2')
        data = zipfile.read()
        open(modelname, 'wb').write(data) 
    landmarkpredictor = dlib.shape_predictor(modelname)

    pspencoder = load_psp_standalone(os.path.join(MODEL_DIR, 'encoder.pt'), device)    
    
vtoonify = VToonify(backbone = 'dualstylegan')
vtoonify.load_state_dict(torch.load(os.path.join(MODEL_DIR, 'cartoon_generator.pt'), map_location=lambda storage, loc: storage)['g_ema'])
vtoonify.to(device)

exstyles = np.load(os.path.join(MODEL_DIR, 'cartoon_exstyle_code.npy'), allow_pickle='TRUE').item()  
styles = []
with torch.no_grad(): 
    for stylename in exstyles.keys():
        exstyle = torch.tensor(exstyles[stylename]).to(device)
        exstyle = vtoonify.zplus2wplus(exstyle)
        styles += [exstyle]
exstyles = torch.cat(styles, dim=0)
    
print('Model successfully loaded!')

## Step 3: Image Toonification

### Visualize and Rescale Input
We rescale the input image to make it fit our pre-trained model

In [ ]:
image_path = './data/077436.jpg'
original_image = load_image(image_path)

In [ ]:
visualize(original_image[0], 30)

In [ ]:
frame = cv2.imread(image_path)
frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

scale = 1
kernel_1d = np.array([[0.125],[0.375],[0.375],[0.125]])
# We detect the face in the image, and resize the image so that the eye distance is 64 pixels.
# Centered on the eyes, we crop the image to almost 400x400 (based on args.padding).
paras = get_video_crop_parameter(frame, landmarkpredictor, padding=[200,200,200,200])
if paras is not None:
    h,w,top,bottom,left,right,scale = paras
    H, W = int(bottom-top), int(right-left)
    # for HR image, we apply gaussian blur to it to avoid over-sharp stylization results
    if scale <= 0.75:
        frame = cv2.sepFilter2D(frame, -1, kernel_1d, kernel_1d)
    if scale <= 0.375:
        frame = cv2.sepFilter2D(frame, -1, kernel_1d, kernel_1d)
    frame = cv2.resize(frame, (w, h))[top:bottom, left:right]
    x = transform(frame).unsqueeze(dim=0).to(device)
else:
    print('no face detected!')

In [ ]:
visualize(x[0].cpu(), 30)

### Select style image

Select the style index (the mapping between index and style image is defined [here](https://github.com/williamyang1991/DualStyleGAN/blob/main/doc_images/cartoon_overview.jpg))

In [ ]:
style_id = [8, 26, 64, 153, 299]

### Style transfer with different cartoon structure styles

In [ ]:
with torch.no_grad():
    I = align_face(frame, landmarkpredictor)
    I = transform(I).unsqueeze(dim=0).to(device)
    s_w = pspencoder(I)
    s_w = vtoonify.zplus2wplus(s_w).repeat(len(style_id), 1, 1)
    s_w[:,:7] = exstyles[style_id,:7]
    x = x.repeat(len(style_id), 1, 1, 1)
    # parsing network works best on 512x512 images, so we predict parsing maps on upsmapled frames
    # followed by downsampling the parsing maps
    x_p = F.interpolate(parsingpredictor(2*(F.interpolate(x, scale_factor=2, mode='bilinear', align_corners=False)))[0], 
                        scale_factor=0.5, recompute_scale_factor=False).detach()
    # we give parsing maps lower weight (1/16)
    inputs = torch.cat((x, x_p/16.), dim=1)
    # d_s has no effect when backbone is toonify
    y_tilde = vtoonify(inputs, s_w, d_s = 0.6)        
    y_tilde = torch.clamp(y_tilde, -1, 1)

In [ ]:
viz = torchvision.utils.make_grid(y_tilde, 5, 2)
visualize(viz.cpu(), 120)

### Navigation with different style degree to achieve flexible style manipulation

Users are suggested to try different style degrees to find the ideal results

In [ ]:
results = []
with torch.no_grad():
    for i in range(5):
        d_s = i / 4.0
        y_tilde = vtoonify(inputs, s_w, d_s = d_s)  
        y_tilde = torch.clamp(y_tilde, -1, 1)
        results += [y_tilde.cpu()]

In [ ]:
vis = torchvision.utils.make_grid(torch.cat(results, dim=0), 5, 2)
visualize(vis, 120)